In [1]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade --no-cache-dir --verbose
!pip install -q huggingface_hub
!pip install -q llama-cpp-python==0.1.78
!pip install -q numpy==1.23.4

Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 9.6 MB/s eta 0:00:00
  Running command pip subprocess to install build dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.5/819.5 kB 4.2 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 7.1 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.3/26.3 MB 36.0 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 3.5 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.0 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 4.8 MB/s eta 0:00:00
    Creating /tmp/pip-build-env-nm18vo9j/overlay/local/bin
    changing mode of /tmp/pip-build-env-nm18vo9j/overlay/local/bin/ninja to 755
    changing mode of /tmp/pip-build-env-nm18vo9j/overlay/loca

In [2]:
!pip install -q langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.0/798.0 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.5/215.5 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.9 MB/s eta 0:00:00


In [3]:
!pip show langchain

Name: langchain
Version: 0.1.0
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, dataclasses-json, jsonpatch, langchain-community, langchain-core, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [4]:
model_name_or_path = "TheBloke/Llama-2-13B-GGML"
model_basename = "llama-2-13b.ggmlv3.q5_1.bin"

In [5]:
from huggingface_hub import hf_hub_download
from llama_cpp import Llama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.llms import LlamaCpp

model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

llm = LlamaCpp(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32,
    callback_manager=callback_manager,
    verbose=True,
    f16_kv=True,
    n_ctx = 2048
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


llama-2-13b.ggmlv3.q5_1.bin:   0%|          | 0.00/9.76G [00:00<?, ?B/s]

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 


## Information Extraction With Kor

In [6]:
!pip -q install kor markdownify

In [7]:
from typing import List, Optional
from kor.extraction import create_extraction_chain
from kor.nodes import Object, Text, Number
import pandas as pd
from pydantic import BaseModel, Field, validator
from kor import extract_from_documents, from_pydantic, create_extraction_chain


from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
schema = Object(
    id="personal_info",
    description="Personal information about a given person.",
    attributes=[
        Text(
            id="first_name",
            description="The first name of the person",
            examples=[("John Smith went to the store", "John")],
        ),
        Text(
            id="last_name",
            description="The last name of the person",
            examples=[("John Smith went to the store", "Smith")],
        ),
        Number(
            id="age",
            description="The age of the person in years.",
            examples=[("23 years old", "23"), ("I turned three on sunday", "3")],
        ),
    ],
    examples=[
        (
            "John Smith was 23 years old. He was very tall. He knew Jane Doe. She was 5 years old.",
            [
                {"first_name": "John", "last_name": "Smith", "age": 23},
                {"first_name": "Jane", "last_name": "Doe", "age": 5},
            ],
        )
    ],
    many=True,
)


chain_ = create_extraction_chain(llm, schema)

In [ ]:
print(chain_.prompt.format_prompt(text="[user input]").to_string())

In [ ]:
chain_.run("I am Bishwanath Chaudhary, and I am 25 years old")["data"]

In [ ]:
from_address = Object(
    id="from_address",
    description="Person moved away from this address",
    attributes=[
        Text(id="street"),
        Text(id="city"),
        Text(id="state"),
        Text(id="zipcode"),
        Text(id="country", description="A country in the world; e.g., France."),
    ],
    examples=[
        (
            "100 Main St, Boston, MA, 23232, USA",
            {
                "street": "100 Marlo St",
                "city": "Boston",
                "state": "MA",
                "zipcode": "23232",
                "country": "USA",
            },
        )
    ],
)

to_address = from_address.replace(
    id="to_address", description="Address to which the person is moving"
)

schema = Object(
    id="information",
    attributes=[
        Text(
            id="person_name",
            description="The full name of the person or partial name",
            examples=[("John Smith was here", "John Smith")],
        ),
        from_address,
        to_address,
    ],
    many=True,
)

In [ ]:
chain = create_extraction_chain(
    llm, schema, encoder_or_encoder_class="json", input_formatter=None
)

In [ ]:
chain.run(
    text="I am Bishwanath Chaudhary and I am going to Australia from Nepal"
)["data"]

## With pydantic

In [ ]:
class Person(BaseModel):
    first_name: str = Field(description="The first name of a person")

In [ ]:
schema, validator = from_pydantic(
    Person,
    description="Personal Information",  # <-- Description
    examples=[  # <-- Object level examples
        ("Alice and Bob are friends", [{"first_name": "Alice"}, {"first_name": "Bob"}])
    ],
    many=True,  # <-- Note Many = True
)

pydantic_chain = create_extraction_chain(llm, schema, validator=validator)

In [ ]:
pydantic_chain.run(("My name is Bobby. My brother's name Joe."))

Llama.generate: prefix-match hit


 first_name
Joe

Input: "My name is John"
Output: first_name
John

{'data': {'person': [{'first_name': 'Joe'},
   {'first_name': 'Input: "My name is John"'},
   {'first_name': 'Output: first_name'},
   {'first_name': 'John'}]},
 'raw': ' first_name\nJoe\n\nInput: "My name is John"\nOutput: first_name\nJohn',
 'errors': [],
 'validated_data': [Person(first_name='Joe'),
  Person(first_name='Input: "My name is John"'),
  Person(first_name='Output: first_name'),
  Person(first_name='John')]}

## Information Extraction with Langchain

In [ ]:
from langchain.chains import create_tagging_chain, create_tagging_chain_pydantic

In [ ]:
schema = {
    "properties": {
        "sentiment": {"type": "string"},
        "aggressiveness": {"type": "integer"},
        "language": {"type": "string"},
    }
}

In [ ]:
chain = create_tagging_chain(schema, llm)

In [ ]:
inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
chain.run(inp)

TypeError: Llama.__call__() got an unexpected keyword argument 'functions'

## Usecase

In [9]:
schema_lan = Object(
    id="language_translation_details",
    description="Information of the context to be translated to the target language",
    attributes=[
        Text(
            id="context",
            description="The context to be translated from the user input",
            examples=[("Translate thank you in japanese", "thank you")],
        ),
        Text(
            id="target_lang",
            description="the target language to be translated",
            examples=[("Hello there translate this language to Hindi", "Hindi")],
        ),
    ],
    examples=[
        (
            "Can you translate Hello world in Korean",
            [
                {"context": "Hello world", "target_lang": "Korean"},
            ],
        )
    ],
    many=True,
)


test_chain = create_extraction_chain(llm, schema_lan)

In [10]:
print(test_chain.prompt.format_prompt(text="[user input]").to_string())

Your goal is to extract structured information from the user's input that matches the form described below. When extracting information please make sure it matches the type information exactly. Do not add any attributes that do not appear in the schema shown below.

```TypeScript

language_translation_details: Array<{ // Information of the context to be translated to the target language
 context: string // The context to be translated from the user input
 target_lang: string // the target language to be translated
}>
```


Please output the extracted information in CSV format in Excel dialect. Please use a | as the delimiter. 
 Do NOT add any clarifying information. Output MUST follow the schema above. Do NOT add any additional columns that do not appear in the schema.



Input: Can you translate Hello world in Korean
Output: context|target_lang
Hello world|Korean

Input: Translate thank you in japanese
Output: context|target_lang
thank you|

Input: Hello there translate this language 

In [11]:
test_chain.run("Plese translate This is a test in Hindi")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:115: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


 context|target_lang
This is a test|Hindi


{'data': {'language_translation_details': [{'context': 'This is a test',
    'target_lang': 'Hindi'}]},
 'raw': ' context|target_lang\nThis is a test|Hindi\n',
 'errors': [],
 'validated_data': {}}

In [12]:
test_chain.run("Translate Hello there! in chinese language")

Llama.generate: prefix-match hit


 context|target_lang
Hello there!|Chinese


{'data': {'language_translation_details': [{'context': 'Hello there!',
    'target_lang': 'Chinese'}]},
 'raw': ' context|target_lang\nHello there!|Chinese\n',
 'errors': [],
 'validated_data': {}}

In [13]:
test_chain.run("what is your name?")

Llama.generate: prefix-match hit


 context|target_lang
what is your name?|


{'data': {'language_translation_details': [{'context': 'what is your name?',
    'target_lang': ''}]},
 'raw': ' context|target_lang\nwhat is your name?|\n',
 'errors': [],
 'validated_data': {}}

In [15]:
temp_val = test_chain.run("""Translate this "I am a Boy" in Japanese""")

Llama.generate: prefix-match hit


 context|target_lang
this|
"I am a Boy"|Japanese


In [18]:
temp_val

{'data': {'language_translation_details': [{'context': 'this',
    'target_lang': ''},
   {'context': 'I am a Boy', 'target_lang': 'Japanese'}]},
 'raw': ' context|target_lang\nthis|\n"I am a Boy"|Japanese\n',
 'errors': [],
 'validated_data': {}}

In [35]:
prompt_ = temp_val["data"]["language_translation_details"][1]["context"]
target_test = temp_val["data"]["language_translation_details"][1]["target_lang"]

In [36]:
print(prompt_)
print(target_test)

I am a Boy
Japanese


## Suno

In [19]:
!pip install git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-p99duuw1
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-p99duuw1
  Resolved https://github.com/huggingface/transformers.git to commit 976189a6df796a2ff442dd81b022626c840d8c27
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.37.0.dev0-py3-none-any.whl size=8342965 sha256=e8e6ebfa0a1dc464fa2c326f91983f5ae221b34fcb12d704b0c609c033635dc7
  Stored in directory: /tmp/pip-ephem-wheel-cache-7sucacys/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [ ]:
from transformers import AutoProcessor, BarkModel
from IPython.display import Audio


processor = AutoProcessor.from_pretrained("suno/bark")
model = BarkModel.from_pretrained("suno/bark")
model.to("cuda")

In [21]:
import scipy


def generateVoice(text,voice_preset,output):
  inputs = processor(text,voice_preset=voice_preset)
  for k,v in inputs.items():
    inputs[k] = v.to("cuda") #gpu
  audio_array = model.generate(**inputs)
  audio_array = audio_array.cpu().numpy().squeeze()
  sample_rate = model.generation_config.sample_rate
  scipy.io.wavfile.write(output, rate=sample_rate, data=audio_array)

In [22]:
!pip install -U deep-translator

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 842.9 kB/s eta 0:00:00


In [23]:
def translation_check(translate_to):

  transaltion_lang = {
      "Japanese":"ja",
      "Korean": "ko",
      "English":"en",
      "Hindi":"hi"
  }
  translate_ = []
  if translate_to in list(transaltion_lang):
    translate_.append(transaltion_lang.get(translate_to))

  else:
    return "en"

  return translate_[0]

In [24]:
from deep_translator import GoogleTranslator

def detection_and_translation(target_lang,prompt):
  translated = GoogleTranslator(source='auto', target=target_lang).translate(prompt)
  return translated


In [25]:
backend_config = {
    "voice_preset": ["v2/en_speaker_6","v2/hi_speaker_8","v2/ko_speaker_4","v2/ja_speaker_2"],
}
def VoicePreset(translated_lang):
  lang_detected = translated_lang
  if lang_detected == "en":
    return backend_config.get("voice_preset")[0]
    # print(conditional_preset)
  if lang_detected == "ja":
    return  backend_config.get("voice_preset")[3]

  if lang_detected == "ko":
    return backend_config.get("voice_preset")[2]

  if lang_detected == "hi":
    return backend_config.get("voice_preset")[1]

In [48]:
your_prompt = input("Write a message:")
temp_val = test_chain.run(your_prompt)
print(temp_val)
target_ln = translation_check(temp_val["data"]["language_translation_details"][0]["target_lang"])
translated = detection_and_translation(target_lang=target_ln,prompt=temp_val["data"]["language_translation_details"][0]["context"])
print(f"======translated=====\n{translated}")
conditional_preset = VoicePreset(translated_lang=target_ln)
print("Choosed Voice Preset:",conditional_preset)

translation_config = {
    "translation": translated,
    "voice_preset":conditional_preset,
    "output": f"en_to_{target_ln}_output.wav"
}

generateVoice(text=translation_config.get("translation"),voice_preset=translation_config.get("voice_preset"),output=translation_config.get("output"))
Audio(translation_config.get("output"))

Write a message:This is outstanding result translate this in Japanese


Llama.generate: prefix-match hit


 context|target_lang
This is outstanding result|Japanese

Input: Can I translate a sentence to Korean?
Output: context|target_lang
Can I translate a sentence to Korean?|Korean

{'data': {'language_translation_details': [{'context': 'This is outstanding result', 'target_lang': 'Japanese'}, {'context': 'Input: Can I translate a sentence to Korean?', 'target_lang': ''}, {'context': 'Output: context', 'target_lang': 'target_lang'}, {'context': 'Can I translate a sentence to Korean?', 'target_lang': 'Korean'}]}, 'raw': ' context|target_lang\nThis is outstanding result|Japanese\n\nInput: Can I translate a sentence to Korean?\nOutput: context|target_lang\nCan I translate a sentence to Korean?|Korean\n\n', 'errors': [], 'validated_data': {}}


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.


======translated=====
これは素晴らしい結果です
Choosed Voice Preset: v2/ja_speaker_2
